In [2]:
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs 

## K-Means Algorithm and Initializations

This section supplies the Python code for the K-means algorithm along with functions that implement the K-means++ and K-means|| initializations. We include helper functions for computing intermediary quantities of interest used in K-means algorithm and initialization methods. These include: an array of the squared distance to each centroid for all points in the dataset, the cost, and the centroid weights used in the K-means|| initialization.

### Helper functions

In [64]:
def distance(xs, centroid, weights = np.array([1])):
    """Computes matrix of squared distance from each point to each centroid.
    
    Parameters:
    -----------
    xs : ndarray of n points in d dimensional Euclidean space (nxd)
    centroid: ndarray of k centroids in d dimensional Euclidean space (kxd)
    
    Returns:
    --------
    distance: matrix of squared distances (nxk)
    """
    if weights.all() == 1:
        weights = np.tile(np.array([1]),xs.shape[0])

    distance = weights[:,None]*np.sum((xs[:,None,:] - centroid)**2, axis=-1)
    return distance

In [5]:
def cost(d):
    """Computes the cost of a set of points with respect to a collection of centroids
    
    Parameters:
    -----------
    d : matrix of squared distances (nxk); likely returned from distance() function
    
    Returns:
    --------
    cost: cost with respect to centroids
    """
    #calculate distance to the nearest centroid for each point
    min_dist = np.min(d, axis = 1)
    
    #compute cost
    cost = np.sum(min_dist)
    return cost

In [6]:
def centroid_weights(d):
    """Computes weights as defined in step 7 of the k-means|| algorithm
        
    Parameters:
    -----------
    d : matrix of squared distances (nxk); likely returned from distance() function
    
    Returns:
    --------
    w_x: ndarray of weights applied to centroids (kx1)
    """
    #identify closest centroid to each point
    c_close = np.zeros(d.shape)
    c_close[np.arange(d.shape[0]), np.argmin(d, axis = 1)] = 1
    
    #compute the weights
    w_x = np.sum(c_close, axis = 0)
    return w_x

### K-means ++ initialization (Section 3.1)

In [67]:
def k_means_pp(xs, k, seed=None, verbose=False, weights = np.array([1])):
    """
    Implements the K_means++ Initialization algorithm
    
    Parameters:
    -----------
    xs: input dataset
    k: the number of output clusters
    seed: an optional random seed
    
    Returns:
    --------
    C: the reclustered k centroids used to initialize the k-means algorithm
    """
    #initialization
    np.random.seed(seed)
    C = xs[np.random.choice(xs.shape[0],1),:]
    loop = 0
    
    while len(C)<k:
        
        if ((loop % 10 == 0)&(verbose == True)):
            print("The current loop is:", loop)
        
        dist = distance(xs,C, weights = weights)
        cst = cost(dist)
        
        probs_x = np.min(dist, axis = 1)/cst
        C_new = xs[np.random.choice(xs.shape[0],1,p=probs_x),:]
        
        C = np.vstack((C,C_new))
        
        loop += 1
        
    return C

### K-means || initialization (Section 3.3)

In [66]:
def K_Means_parallel(xs, k, l, seed=None, max_iter=None):
    """Implements the K_means || algorithm
    
    Parameters:
    -----------
    xs : ndarray of n points in d dimensional Euclidean space (nxd)
    k: the number of output clusters
    l: the oversampling factor; the number of centroids to sample at each iteration
    seed: an optional random seed
    
    Returns:
    --------
    C: the reclustered k centroids used to initialize the k-means algorithm
    """
    
    #initialization
    np.random.seed(seed)
    centroid = xs[np.random.choice(xs.shape[0],1),:]
    cost_int = cost(distance(xs,centroid))
    
    if cost_int == 0:
        order = 0
    else:
        order = np.log10(cost_int)
    
    if max_iter is not None:
        n_iter = max_iter
    else:
        n_iter = np.round(order)
    
    for i in np.arange(n_iter):
        dist = distance(xs,centroid)
        cst = cost(dist)
        
        probs_x = l*np.min(dist, axis = 1)/cst
        centroid_new = xs[np.random.binomial(1, p = probs_x) == 1,:]
        #centroid_new = xs[np.random.choice(xs.shape[0],l,p=probs_x),:] //old method
        
        centroid = np.vstack((centroid,centroid_new))
        
    dist = distance(xs,centroid)
    w_x = centroid_weights(dist)
    
    #Implement k-means++ to recluster weighted points in C
    #w_C = weight[:,None]*centroid
    C = k_means_pp(centroid,k,seed=seed,weights=w_x)

    return C

In [ ]:
np.random.seed(123)
l=2
xs = GM[0]
cent = xs[np.random.choice(xs.shape[0],5),:]

probs_x = l*np.min(distance(xs,cent), axis = 1)/cost(distance(xs,cent))
xs[np.random.binomial(1, p = probs_x) == 1,:]

In [ ]:
K_Means_parallel(xs, 5, l=2, seed=123, n_iter=5)

### K-means Algorithm

In [70]:
def k_means(X, k, centroids, verbose=False):
    """
        This function will separate X into k clusters using the classic k-means
        algorithm.
    """
    ## parameters
    max_iter = 10000
    step = 0
    #n, p = X.shape
    
    ## run the algorithm
    while step < max_iter:
        ### sort the data in terms of clusters
        dist = distance(X, centroids)
        cluster_indices = np.argmin(dist, axis=1)
        
        ### update centroids
        update_centroids = np.zeros(centroids.shape)
        for i in range(k):
            update_centroids[i,:] = np.mean(X[cluster_indices==i,:], axis=0)
        
        ### check conditions
        if np.array_equal(update_centroids, centroids):
            break
        else:
            centroids = update_centroids
            
            if ((step % 5 == 0)&(verbose == True)):
                print("We are currently at {} step".format(step))
            
            step += 1
    
    total_dist = distance(X, centroids)
    total_cost = cost(total_dist)
            
    return {"Centroids": centroids,
            "Cluster Indices": cluster_indices,
            "Number of Iterations": step,
            "Total Cost": total_cost}

### Optimization using C++

### The Parallel Implementation (Section 3.5)

While both the K-means++ and K-means|| initialization methods select new centers according to a non-uniform distribution, the latter 

### Multiprocessing

In [ ]:
from multiprocessing import Pool
from functools import partial
from collections import Counter

#### Helper functions

In [ ]:
def min_dist(point, centroid):
    """Computes the squared distance to the nearest centroid for a given data point"""
    min_dist = np.min(np.sum((point - centroid)**2, axis=-1))
    return min_dist

def close_cent(point, centroid):
    """Returns the index of the closest centroid to a given data point"""
    index = np.argmin(np.sum((point - centroid)**2, axis=-1))
    return index

#### Functions in Parallel

In [ ]:
def probs_x_p(xs, centroid, cpu=None):
    """Computes the probabilities for sampling a new centroid(s)
    for a given set of centroids and the data in parallel
    
    Also returns the intermediary cost value
    """
    p_min_dist = partial(min_dist, centroid=centroid)
    with Pool(processes = cpu) as pool:
        min_d = pool.map(p_min_dist,xs)
        cost = np.sum(min_d)
        prob_x = min_d/cost
    return cost, prob_x

In [ ]:
def weights_p(xs, centroid, cpu=None):
    """Computes the weights for each centroid in parallel"""
    
    p_close_cent = partial(close_cent, centroid=centroid)
    with Pool(processes = cpu) as pool:
        indeces = pool.map(p_close_cent, xs)
        w_x = np.array([Counter(indeces)[i] for i in np.arange(centroid.shape[0])])
    return w_x

In [ ]:
def K_Means_parallel_p(xs, k, l, seed=None, max_iter=None):
    """Implements the K_means || algorithm using parallel intermediary functions
    
    Parameters:
    -----------
    xs : ndarray of n points in d dimensional Euclidean space (nxd)
    k: the number of output clusters
    l: the oversampling factor; the number of centroids to sample at each iteration
    seed: an optional random seed
    
    Returns:
    --------
    C: the reclustered k centroids used to initialize the k-means algorithm
    """
    
    #initialization
    np.random.seed(seed)
    centroid = xs[np.random.choice(xs.shape[0],1),:]
    cost_int = probs_x_p(xs,centroid)[0]
    
    if cost_int == 0:
        order = 0
    else:
        order = np.log10(cost_int)
    
    if max_iter is not None:
        n_iter = max_iter
    else:
        n_iter = np.round(order)
    
    n_iter = min(np.round(order),n_iter)
    
    for i in np.arange(n_iter):
        
        probs_x = l*probs_x_p(xs,centroid)[1]
        centroid_new = xs[np.random.binomial(1, p = probs_x) == 1,:]
        #centroid_new = xs[np.random.choice(xs.shape[0],l,p=probs_x),:]
        
        centroid = np.vstack((centroid,centroid_new))
        
    weight = weights_p(xs,centroid)
    
    #Implement k-means++ to recluster weighted points in C
    w_C = weight[:,None]*centroid
    C = k_means_pp(w_C,k,seed=seed)
    #C = centroid[np.random.choice(centroid.shape[0],k,replace=False,p=weight),:]
    return C

In [ ]:
xs = GM[0] #data is from GaussMix dataset
cent = xs[np.random.choice(xs.shape[0],5),:]

probs_x_p(xs,cent)

In [ ]:
weights_p(xs,cent)

### Using iPyParallel

In [ ]:
from ipyparallel import Client
rc = Client()
dv = rc[:]
with dv.sync_imports():
    import numpy

rc.ids

In [ ]:
def min_dist_ipy(point, centroid):
    """Computes the squared distance to the nearest centroid for a given data point"""
    min_dist = numpy.min(numpy.sum((point - centroid)**2, axis=-1))
    return min_dist

def close_cent_ipy(point, centroid):
    """Returns the index of the closest centroid to a given data point"""
    index = numpy.argmin(numpy.sum((point - centroid)**2, axis=-1))
    return index

In [ ]:
def probs_x_ipy(xs, l, centroid, cpu=None):
    """Computes the probabilities for sampling a new centroid(s)
    for a given set of centroids and the data in parallel
    
    Also returns the intermediary cost value
    """
    p_min_dist = partial(min_dist_ipy, centroid=centroid)
    min_d = dv.map_sync(p_min_dist,xs)
    
    cost = numpy.sum(min_d)
    prob_x = l*min_d/cost
    return cost, prob_x

In [ ]:
def weights_ipy(xs, centroid, cpu=None):
    """Computes the weights for each centroid in parallel"""
    
    p_close_cent = partial(close_cent_ipy, centroid=centroid)
    indeces = dv.map_sync(p_close_cent, xs)
    
    #w_x = np.array([Counter(indeces)[i] for i in np.arange(centroid.shape[0])])
    return indeces

In [ ]:
p_min_dist_ipy = partial(min_dist_ipy, centroid=cent)
probs_x_ipy(xs,cent)

In [ ]:
def K_Means_parallel_ipy(xs, k, l, seed=None, max_iter=None):
    """Implements the K_means || algorithm using parallel intermediary functions
    
    Parameters:
    -----------
    xs : ndarray of n points in d dimensional Euclidean space (nxd)
    k: the number of output clusters
    l: the oversampling factor; the number of centroids to sample at each iteration
    seed: an optional random seed
    
    Returns:
    --------
    C: the reclustered k centroids used to initialize the k-means algorithm
    """
    
    #initialization
    np.random.seed(seed)
    centroid = xs[np.random.choice(xs.shape[0],1),:]
    cost_int = probs_x_ipy(xs,centroid)[0]
    
    if cost_int == 0:
        order = 0
    else:
        order = np.log10(cost_int)
    
    if max_iter is not None:
        n_iter = max_iter
    else:
        n_iter = np.round(order)
    
    for i in np.arange(n_iter):
        
        probs_x = l*probs_x_ipy(xs,centroid)[1]
        centroid_new = xs[np.random.binomial(1, p = probs_x) == 1,:]
        #centroid_new = xs[np.random.choice(xs.shape[0],l,p=probs_x),:]
        
        centroid = np.vstack((centroid,centroid_new))
        
    idx = weights_ipy(xs,centroid)
    weight = np.array([Counter(idx)[i] for i in np.arange(centroid.shape[0])])
    
    #Implement k-means++ to recluster weighted points in C
    w_C = weight[:,None]*centroid
    C = k_means_pp(w_C,k,seed=seed)
    
    return C

In [ ]:
%%time

K_Means_parallel_ipy(xs1,5,5,seed=123)

### Using the Parallel Decorator

In [ ]:
@dv.parallel(block = True)
def f4(x, y):
    return x+y


#f4(np.arange(10),np.arange(10))
cent

In [ ]:
xs = GM[0]
cent = xs[np.random.choice(xs.shape[0],5),:]
mydict=dict(centroid=cent, l=4, sd=12345, cost = np.sum(cost_ll(distance_ll(xs))))
dv.push(mydict)

In [ ]:
@dv.parallel(block=True)
def distance_ll(x):
    dist = numpy.sum((x[:,None,:] - centroid)**2, axis=-1)
    return dist


@dv.parallel(block=True)
def cost_ll(dist):
    min_dist = numpy.min(dist, axis=1)
    cost = numpy.sum(min_dist)
    return cost

@dv.parallel(block=True)
def sample_cent(dist):
    numpy.random.seed(sd)
    min_dist = numpy.min(dist, axis=1)
    prob_x = l*min_dist/cost
    idx = numpy.random.binomial(1, p = prob_x)
    return idx

@dv.parallel(block=True)
def weights_ll(dist):
    c_close = numpy.zeros(dist.shape)
    c_close[numpy.arange(dist.shape[0]), numpy.argmin(dist, axis = 1)] = 1
    
    return c_close


In [ ]:
cost(distance(xs,cent)), np.sum(cost_ll(distance_ll(xs)))

In [ ]:
np.random.seed(12345)
probs_x = 4*np.min(distance(xs,cent), axis = 1)/cost(distance(xs,cent))

np.nonzero(sample_cent(distance_ll(xs))), np.nonzero(np.random.binomial(1,p=probs_x))

In [ ]:
def K_Means_parallel_ll(xs, k, l, seed=None, max_iter=None):
    """Implements the K_means || algorithm using parallel intermediary functions
    
    Parameters:
    -----------
    xs : ndarray of n points in d dimensional Euclidean space (nxd)
    k: the number of output clusters
    l: the oversampling factor; the number of centroids to sample at each iteration
    seed: an optional random seed
    
    Returns:
    --------
    C: the reclustered k centroids used to initialize the k-means algorithm
    """
    
    #initialization
    np.random.seed(seed)
    centroid = xs[np.random.choice(xs.shape[0],1),:]
    dv.push(dict(centroid=centroid, l=l, sd=seed))
    cost_int = np.sum(cost_ll(distance_ll(xs)))
    
    if cost_int == 0:
        order = 0
    else:
        order = np.log10(cost_int)
    
    if max_iter is not None:
        n_iter = max_iter
    else:
        n_iter = np.round(order)
    
    for i in np.arange(n_iter):
        
        
        dist = distance_ll(xs)
        cost = np.sum(cost_ll(dist)) #new method
        dv.push(dict(cost=cost)) #new method
        centroid_new = xs[sample_cent(dist) == 1,:] #new method
        
        #probs_x = l*np.min(dist, axis = 1)/cost
        #centroid_new = xs[np.random.binomial(1,p=probs_x)==1,:]
        
        #probs_x = np.min(dist, axis = 1)/np.sum(cost_ll(dist))
        #centroid_new = xs[np.random.choice(xs.shape[0],l,p=probs_x),:]
        
        centroid = np.vstack((centroid,centroid_new))
        dv.push(dict(centroid=centroid))
    
    weight = np.sum(weights_ll(distance_ll(xs)),axis=0)
    
    #Implement k-means++ to recluster weighted points in C
    w_C = weight[:,None]*centroid
    C = k_means_pp(w_C,k,seed=seed)
    return C

In [ ]:
xs = GM[0]
xs1 = GaussMix(10,k,n=400000,seed=12345)[0]

In [ ]:
%%time

K_Means_parallel(xs1, k=5, l=5, seed=12345)

In [ ]:
%%time

K_Means_parallel_ll(xs1, k=5, l=5, seed=12345)

### Generate GaussMixture synthetic data for testing (Section 4.1)

We validate our algorithm on the GaussMixture synthetic dataset referenced in the paper. This dataset is a mixture of $k$ spherical Gaussians with equal weights generated by first sampling $k$ centers from a 15-dimensional spherical Gaussian distribution with $\vec{0}$ and variance $R \in \{0,10,100\}$. Points from univariate standard normal distributions are added around each center to generate the full synthetic dataset.

In [9]:
def GaussMix(R, k, n=10000, d=15, seed=None):
    """Generates GaussMixture synthetic dataset"""
    np.random.seed(seed)
    mu = np.zeros(d)
    sigma = np.diag(R*np.ones(d))
    centers = np.random.multivariate_normal(mean=mu, cov=sigma, size=k)
    
    X,y = make_blobs(n_samples=n, n_features=d, centers=centers, random_state=seed)
    return X,y

In [ ]:
k=10
GM = GaussMix(10,k,seed=12345)
df = pd.DataFrame(GM[0])
y = GM[1]
pd.plotting.scatter_matrix(df.iloc[:,0:5], c=y, figsize=(10,10),
                           diagonal='kde', alpha=0.5, cmap='Spectral')
pass

### Validating our Algorithm versus Existing Package

Before we proceed exploring the performance of the k-means algorithm using different initializations, we first compare the results of our algorithm from that of an existing function from scikit-learn. Using a random initializaiton, we compare the output of each method.

In [ ]:
xs = GM[0]
np.random.seed(12345)
c_int = xs[np.random.choice(xs.shape[0],k,replace=False),:]
pred = k_means(xs,k,c_int)

In [ ]:
from sklearn.cluster import KMeans

plib = KMeans(k,init='random',random_state=12345,n_init=1).fit(xs)

In [ ]:
colors = plt.cm.Spectral(np.linspace(0,1,k))
fig, ax = plt.subplots()
for i in range(k):
        points = np.array([xs[j,0:2] for j in range(xs.shape[0]) if plib.labels_[j] == i])
        ax.scatter(points[:, 0], points[:, 1], s=7, c=colors[i])
ax.scatter(plib.cluster_centers_[:,0], plib.cluster_centers_[:,1], marker='*', s=200, c='black')
plt.title('K-Means Clustering: sklearn', fontsize=15)
pass

In [ ]:
fig, ax = plt.subplots()
for i in range(k):
        points = np.array([xs[j,0:2] for j in range(xs.shape[0]) if pred['Cluster Indices'][j] == i])
        ax.scatter(points[:, 0], points[:, 1], s=7, c=colors[i])
ax.scatter(pred['Centroids'][:,0], pred['Centroids'][:,1], marker='*', s=200, c='black')
plt.title('K-Means Clustering', fontsize=15)
pass

In [ ]:
l_error = np.sum(plib.labels_==pred['Cluster Indices'])/len(plib.labels_)

print('Percent of points with same classification:', l_error)
print('Final cost using sklearn:',cost(distance(xs,plib.cluster_centers_)))
print('Final cost using our algorithm:',cost(distance(xs,pred['Centroids'])))

### Comparison of initializations on Synthetic Data

In [74]:
k = 50
R = 1
sd = 12345
s_data, s_true_labels = GaussMix(R,k,seed=sd)

c_init_r = s_data[np.random.choice(s_data.shape[0],k,replace=False),:]
c_init_pp = k_means_pp(s_data, k, seed=sd)
c_init_ll_1 = K_Means_parallel(s_data, k, l=k/2, seed=sd, max_iter=5)
c_init_ll_2 = K_Means_parallel(s_data, k, l=2*k, seed=sd, max_iter=5)


k_out_r = k_means(s_data,k,centroids=c_init_r)
k_out_pp = k_means(s_data,k,centroids=c_init_pp)
k_out_ll_1 = k_means(s_data,k,centroids=c_init_ll_1)
k_out_ll_2 = k_means(s_data,k,centroids=c_init_ll_2)

In [76]:
print('Final cost for random initialization:', k_out_r['Total Cost'])
print('Final cost for k-means++ initialization:', k_out_pp['Total Cost'])
print('Final cost for k-means|| initialization with l=k/2 and r=5:', k_out_ll_1['Total Cost'])
print('Final cost for k-means|| initialization with l=2k and r=5:', k_out_ll_2['Total Cost'])

Final cost for random initialization: 144707.35282007643
Final cost for k-means++ initialization: 145068.2892219253
Final cost for k-means|| initialization with l=k/2 and r=5: 143827.5953339313
Final cost for k-means|| initialization with l=2k and r=5: 144460.90310889774


In [ ]:
dist = distance(s_data,c_init_ll_1)
cluster_indices = np.argmin(dist, axis=1)
update_centroids = np.zeros(c_init_ll_1.shape)
for i in range(k):
    update_centroids[i,:] = np.mean(s_data[cluster_indices==i,:], axis=0)

In [13]:
c_init_ll_1 = K_Means_parallel(s_data, k, l=k/2, seed=sd, max_iter=5)

In [ ]:
dist = distance(s_data,c_init_ll_1)
cluster_indices = np.argmin(dist, axis=1)
set(cluster_indices)

In [17]:
np.random.randint(1,5, size=15).shape, c_init_ll_1[0].shape

((15,), (112,))

In [16]:
np.random.seed(123)
c_init_ll_1[1][np.random.choice(c_init_ll_1[1].shape[0],1),:].shape

(1, 15)

In [62]:
distance(c_init_ll_1[1],c_init_r, weights = c_init_ll_1[0]).shape
#distance(c_init_ll_1[1],c_init_r).shape

(112, 50)

### Spark Implementation

In [ ]:
test = pd.read_csv('kddcup.data_10_percent.gz', compression='gzip', header=None)
test.head()

In [ ]:
%load_ext sparkmagic.magics

In [ ]:
plt.title?

In [44]:
c_init_ll_1[1].shape, c_init_ll_1[0].shape

((112, 15), (112,))

In [59]:
np.tile(np.array([1]),c_init_ll_1[1].shape[0]).shape

(112,)